In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%pprint
%matplotlib inline
from IPython import display
from PIL import Image, ImageFont, ImageDraw
from math import cos, sin, pi, sqrt, atan
from matplotlib.pyplot import imshow
from pathlib import Path
import imageio
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import requests
import sys

ccw_history_radius_dict = load_object('ccw_history_radius_dict')
ccw_history_theta_dict = load_object('ccw_history_theta_dict')
ccw_history_year_dict = load_object('ccw_history_year_dict')
cw_history_radius_dict = load_object('cw_history_radius_dict')
cw_history_theta_dict = load_object('cw_history_theta_dict')
cw_history_year_dict = load_object('cw_history_year_dict')
dresses_file_dict = load_object('dresses_file_dict')
dresses_list = load_object('dresses_list')
first_date_dict = load_object('first_date_dict')
generations_df = load_object('generations_df')
patriline_df = load_object('patriline_df')
saeculum_cmap_dict = load_object('saeculum_cmap_dict')
second_date_dict = load_object('second_date_dict')
turnings_df = load_object('turnings_df')

MAX_HEIGHT = 936
MAX_WIDTH = 936
dresses_regex = re.compile(r'(\d{4})-(\d{4})\.png')
jpg_dir = os.path.join(SAVES_FOLDER, 'jpg')
min_year = patriline_df['year_of_birth'].min()
movie_folder = os.path.join(SAVES_FOLDER, 'movies')
bare_movie_folder = os.path.join(movie_folder, 'bare')
png_folder = os.path.join(SAVES_FOLDER, 'png')
saeculum_dir = os.path.join(DATA_FOLDER, 'saeculum')
saeculum_movie_folder = os.path.join(movie_folder, 'saeculum')
saeculum_fashionable_movie_folder = os.path.join(movie_folder, 'saeculum_fashionable')

white_tuple = (255, 255, 255, 0)
black_tuple = (0, 0, 0, 255)
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 Image	 ImageDraw	 ImageFont	 MAX_HEIGHT	 MAX_WIDTH	 Path	 
SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 atan	 attempt_to_pickle	 bare_movie_folder	 black_tuple	 ccw_history_radius_dict	 ccw_history_theta_dict	 
ccw_history_year_dict	 cos	 csv	 cw_history_radius_dict	 cw_history_theta_dict	 cw_history_year_dict	 display	 dresses_file_dict	 dresses_list	 
dresses_regex	 example_iterrows	 first_date_dict	 generations_df	 get_column_descriptions	 imageio	 imshow	 jpg_dir	 load_csv	 
load_dataframes	 load_object	 logging	 math	 min_year	 movie_folder	 np	 os	 patriline_df	 
pd	 pi	 pickle	 plt	 png_folder	 re	 requests	 saeculum_cmap_dict	 saeculum_dir	 
saeculum_fashionable_movie_folder	 saeculum_movie_folder	 save_dataframes	 second_date_dict	 sin	 sqrt	 store_objects	 sys	 turnings_df	 
white_tuple	 


In [2]:

def polar_to_cartesian(r, theta):
    radians = theta*(pi/180)
    
    return int(r*cos(radians)), int(r*sin(radians))

In [3]:

def create_xy_list(history_radius_dict):
    xy_list = []
    for radius in sorted(history_radius_dict.keys()):
        year, theta = history_radius_dict[radius]
        cartesian_tuple = polar_to_cartesian(radius, theta)
        if len(xy_list):
            if (cartesian_tuple != xy_list[-1]):
                xy_list.append(cartesian_tuple)
        else:
            xy_list.append(cartesian_tuple)
    
    return xy_list

In [4]:

def get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i=0):
    xy_list = []
    i = i % 4
    if stop_year > stopped_year:
        stop_year = stopped_year
    start_radius = history_year_dict[start_year][0]
    start_radius += 25*i
    stop_radius = history_year_dict[stop_year][0]
    stop_radius += 25*i
    increment_count = int(2*pi*start_radius)
    radius_array = np.linspace(start=start_radius, stop=stop_radius,
                               num=increment_count)
    start_theta = history_year_dict[start_year][1]
    stop_theta = history_year_dict[stop_year][1]
    theta_array = np.linspace(start=start_theta, stop=stop_theta,
                              num=increment_count)
    for radius, theta in zip(radius_array, theta_array):
        cartesian_tuple = polar_to_cartesian(radius, theta)
        if len(xy_list):
            if (cartesian_tuple != xy_list[-1]):
                xy_list.append(cartesian_tuple)
        else:
            xy_list.append(cartesian_tuple)
    
    return xy_list

In [5]:

def show_year_image(year):
    file_name = '{}.jpg'.format(year)
    file_path = os.path.join(jpg_dir, file_name)
    jpg_image = Image.open(fp=file_path, mode='r')
    jpg_image = jpg_image.rotate(angle=180)
    width, height = jpg_image.size
    if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
        if (width > MAX_WIDTH):
            multiple = MAX_WIDTH / width
            width *= multiple
            height *= multiple
        if (height > MAX_HEIGHT):
            multiple = MAX_HEIGHT / height
            width *= multiple
            height *= multiple
        jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [6]:

def show_turning_image(year):
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    turning_name_list = turnings_df[match_series].index.tolist()
    if len(turning_name_list):
        turning_name = turning_name_list[0]
        file_name = '{}.jpg'.format(turning_name)
        file_path = os.path.join(jpg_dir, file_name)
        jpg_image = Image.open(fp=file_path, mode='r')
        jpg_image = jpg_image.rotate(angle=180)
        width, height = jpg_image.size
        if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
            if (width > MAX_WIDTH):
                multiple = MAX_WIDTH / width
                width *= multiple
                height *= multiple
            if (height > MAX_HEIGHT):
                multiple = MAX_HEIGHT / height
                width *= multiple
                height *= multiple
            jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [7]:

def show_saeculum_image(saeculum_name):
    file_name = '{}.jpg'.format(saeculum_name)
    file_path = os.path.join(jpg_dir, file_name)
    jpg_image = Image.open(fp=file_path, mode='r')
    jpg_image = jpg_image.rotate(angle=180)
    width, height = jpg_image.size
    if (width > MAX_WIDTH) or (height > MAX_HEIGHT):
        if (width > MAX_WIDTH):
            multiple = MAX_WIDTH / width
            width *= multiple
            height *= multiple
        if (height > MAX_HEIGHT):
            multiple = MAX_HEIGHT / height
            width *= multiple
            height *= multiple
        jpg_image = jpg_image.resize(size=(int(width), int(height)), resample=0, box=None)
        '''
        width, height = jpg_image.size
    left = 0 - int(width/2)
    right = int(width/2)
    top = 0 - int(height/2)
    bottom = int(height/2)
    AxesImage_obj = imshow(X=np.asarray(jpg_image), origin='upper', extent=(left, right, bottom, top))'''
    
    return jpg_image

In [8]:

def add_fashion_image(year):
    
    # Get old image data
    old_path = os.path.join(png_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    old_data_list = foreground.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        if (old_tuple[0] == 255) and (old_tuple[1] == 255) and (old_tuple[2] == 255):
            new_data_list.append(white_tuple)
        else:
            new_data_list.append(old_tuple)
    
    # Replace old with new
    foreground.putdata(new_data_list)
    
    # Get dresses image
    dresses_file = dresses_file_dict[year]
    if dresses_file is None:
        file_path = os.path.join(png_folder, 'plot_{}_fashionable.png'.format(year))
        foreground.save(file_path, 'PNG')
    else:
        file_path = os.path.join(png_folder, dresses_file)
        dresses_img = Image.open(file_path)
        dresses_img = dresses_img.convert('RGBA')

        dresses_img.paste(foreground, (0, 0), foreground)

        file_path = os.path.join(png_folder, 'plot_{}_fashionable.png'.format(year))
        dresses_img.save(file_path, 'PNG')

In [9]:

def add_saeculum_image(year):
    
    # Get old image data
    old_path = os.path.join(bare_movie_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    old_data_list = foreground.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        if (old_tuple[0] == 255) and (old_tuple[1] == 255) and (old_tuple[2] == 255):
            new_data_list.append(white_tuple)
        else:
            new_data_list.append(old_tuple)
    
    # Replace old with new
    foreground.putdata(new_data_list)
    
    # Get saeculum image
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    saeculum_list = turnings_df[match_series].index.tolist()
    if len(saeculum_list):
        saeculum_name = saeculum_list[0]
        saeculum_file = '{}.png'.format(saeculum_name)
        saeculum_path = Path(os.path.join(saeculum_dir, saeculum_file))
        new_path = os.path.join(saeculum_movie_folder, 'plot_{}_saeculum.png'.format(year))
        if saeculum_path.is_file():
            saeculum_img = Image.open(saeculum_path, mode='r')
            saeculum_img = saeculum_img.convert('RGBA')
            saeculum_img.paste(foreground, (0, 0), foreground)
            saeculum_img.save(new_path, 'PNG')
        else:
            foreground.save(new_path, 'PNG')

In [27]:

def show_stopped_babbitt_plot(history_year_dict, history_theta_dict, stopped_year):

    # Turn interactive plotting off
    plt.ioff()
    
    # Create a new figure, plot into it, then close it so it never gets displayed
    fig = plt.figure(figsize=(13, 13))
    ax = fig.add_subplot(111, autoscale_on=False)
    ax.set_xlim(-1000, 1000)
    ax.set_ylim(-1000, 1000)
    match_series = (patriline_df['year_of_birth'] <= stopped_year)
    i = patriline_df[match_series].shape[0]-1
    d = 5
    previous_saeculum = patriline_df[match_series].head(1)['saeculum_name'].tolist()[0]
    for patriarch_name, row_series in patriline_df[match_series].iterrows():
        start_year = int(row_series['year_of_birth'])
        stop_year = row_series['year_of_death']
        try:
            stop_year = int(stop_year)
        except:
            stop_year = start_year + 80
            if stop_year > max(history_year_dict.keys()):
                stop_year = max(history_year_dict.keys())
        xy_list = get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i)
        years_list = [year for year in [start_year, stop_year] if year <= stopped_year]
        add_spiral_labels(years_list, history_year_dict, i)
        if stop_year > stopped_year:
            stop_year = stopped_year
        text_color = row_series['xkcd_color']
        label_arc(start_year, stop_year, history_year_dict, history_theta_dict, patriarch_name, 13, i, text_color)
        saeculum = row_series['saeculum_name']
        if saeculum != previous_saeculum:
            previous_saeculum = saeculum
            d = 5
        #print(patriarch_name, i, d, saeculum)
        cmap = saeculum_cmap_dict[saeculum]
        c = plt.get_cmap(cmap)(np.linspace(0, 1, 6))[d]
        PathCollection_obj = plt.plot([xy[0] for xy in xy_list], [xy[1] for xy in xy_list],
                                      alpha=0.75, label=patriarch_name, c=c, linewidth=9)
        i -= 1
        d -= 1
    
    #show_turning_image(stopped_year)
    legend_obj = ax.legend()
    frame_obj = legend_obj.get_frame()
    frame_obj.set_facecolor('whitesmoke')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # Close it so it never gets displayed
    file_name = 'plot_{}.png'.format(stopped_year)
    file_path = os.path.join(bare_movie_folder, file_name)
    plt.savefig(file_path, format='png')
    plt.close(fig)

In [11]:

def add_spiral_labels(years_list, history_year_dict, i=0):
    i = i % 4
    for year in years_list:
        radius, theta = history_year_dict[year]
        radius += 25*i
        radius -= 25/2
        x, y = polar_to_cartesian(radius, theta)
        text_obj = plt.text(x, y, year, fontsize=10, color='gray',
                            rotation=theta-90, rotation_mode='anchor')

In [12]:

def label_arc(start_year, stopped_year, history_year_dict, history_theta_dict,
              arc_label, ideal_distance=13, i=0, label_color='black'):
    i = i % 4
    starting_year = int(((start_year + stopped_year) / 2) - (len(arc_label) / 2))
    starting_radius, starting_theta = history_year_dict[starting_year]
    next_radius, next_theta = history_year_dict[starting_year+1]
    

    # Tan(A) = Opposite/Adjacent
    radians = atan(ideal_distance/starting_radius)
    theta_sign = np.sign(next_theta-starting_theta)
    
    # Increment the theta so that it spaces the letters the same regardless of the radius
    theta_increment = theta_sign*(radians*180/pi)
    
    # Figure out if you have to flip the characters upside-down and place them backwards
    if (starting_theta%360) < 200:
        logging.info('')
        logging.info('Right-side up thetas:')
        theta = starting_theta
        radius = starting_radius + 25*i
        radius -= 25/2
        for c in arc_label[::int(-theta_sign)]:
            logging.info('c: "{}", radius: "{}", theta: "{}"'.format(c, radius, theta % 360))
            x, y = polar_to_cartesian(radius, theta)
            text_obj = plt.text(x, y, c, fontsize=12, color=label_color,
                                rotation=theta-90, rotation_mode='anchor')
            theta += theta_increment
            if int(theta) in history_theta_dict:
                radius = history_theta_dict[int(theta)][1]
                radius += 25*i
                radius -= 25/2
    else:
        logging.info('')
        logging.info('Upside-down thetas:')
        theta = starting_theta + theta_increment*len(arc_label)
        radius = starting_radius + 25*i
        radius += 25/2
        for c in arc_label[::int(-theta_sign)]:
            logging.info('c: "{}", radius: "{}", theta: "{}"'.format(c, radius, theta % 360))
            x, y = polar_to_cartesian(radius, theta)
            text_obj = plt.text(x, y, c, fontsize=12, color=label_color,
                                rotation=theta+90, rotation_mode='anchor')
            theta -= theta_increment
            if int(theta) in history_theta_dict:
                radius = history_theta_dict[int(theta)][1]
                radius += 25*i
                radius += 25/2

In [13]:

def distance_from_white(old_tuple):
    x_diff = 255 - old_tuple[0]
    y_diff = 255 - old_tuple[1]
    z_diff = 255 - old_tuple[2]
    
    return sqrt(x_diff**2 + y_diff**2 + z_diff**2)

In [14]:

def distance_from_black(old_tuple):
    
    return sqrt(old_tuple[0]**2 + old_tuple[1]**2 + old_tuple[2]**2)

In [15]:

def make_transparent(img, threshold=38):
    margin = 255/distance_from_white(black_tuple)
    old_data_list = img.getdata()
    
    # Get new image data
    new_data_list = []
    for old_tuple in old_data_list:
        transparency = int(margin * distance_from_white(old_tuple))
        if transparency > threshold:
            transparency = 255
        elif transparency < 0:
            transparency = 0
        old_tuple = (old_tuple[0], old_tuple[1], old_tuple[2], transparency)
        new_data_list.append(old_tuple)
    
    # Replace old with new
    img.putdata(new_data_list)
    
    return img

In [16]:

def add_fashion_saeculum_image(year):
    
    # Get old image data
    old_path = os.path.join(bare_movie_folder, 'plot_{}.png'.format(year))
    foreground = Image.open(old_path)
    foreground = foreground.convert('RGBA')
    foreground = make_transparent(foreground, threshold=6)
    
    # Get dresses image
    dresses_file = dresses_file_dict[year]
    if dresses_file is None:
        dresses_img = foreground
    else:
        file_path = os.path.join(png_folder, dresses_file)
        dresses_img = Image.open(file_path)
        dresses_img = dresses_img.convert('RGBA')
        dresses_img = make_transparent(dresses_img, threshold=5)
        dresses_img.paste(foreground, (0, 0), foreground)
    
    # Get saeculum image
    match_series = (turnings_df['turning_begin_year'] <= year) & (turnings_df['turning_end_year'] >= year)
    saeculum_list = turnings_df[match_series].index.tolist()
    if len(saeculum_list):
        saeculum_name = saeculum_list[0]
        saeculum_file = '{}.png'.format(saeculum_name)
        saeculum_path = Path(os.path.join(saeculum_dir, saeculum_file))
        if saeculum_path.is_file():
            saeculum_img = Image.open(saeculum_path, mode='r')
            saeculum_img = saeculum_img.convert('RGBA')
            saeculum_img.paste(dresses_img, (0, 0), dresses_img)
        else:
            saeculum_img = dresses_img
        file_path = os.path.join(saeculum_fashionable_movie_folder, 'plot_{}_saeculum_fashionable.png'.format(year))
        saeculum_img.save(file_path, 'PNG')

In [17]:

def make_a_movie(stop_year=2020, png_suffix='', movie_folder=bare_movie_folder, movie_name=None):
    images_list = []
    for stopped_year in range(min_year, stop_year):
        png_name = 'plot_{}{}.png'.format(stopped_year, png_suffix)
        png_path = os.path.join(movie_folder, png_name)
        images_list.append(imageio.imread(png_path))
    png_name = 'plot_{}{}.png'.format(stop_year, png_suffix)
    png_path = os.path.join(movie_folder, png_name)
    for i in range(200):
        images_list.append(imageio.imread(png_path))
    if movie_name is None:
        movie_name = '_'.join(png_suffix.split('_')[1:] + ['movie']) + '.gif'
    gif_path = os.path.join(movie_folder, movie_name)
    imageio.mimsave(gif_path, images_list)

In [18]:

def exists(path):
    r = requests.head(path)
    
    return r.status_code == requests.codes.ok

In [19]:

patriline_df.sample(5).T

patriarch_name,"Elkanah Babbitt, I",William David Babbitt,"Elkanah Babbitt, III",David George Babbitt,Stephen Elkanah Babbitt
year_of_birth,1665,1791,1710,1961,1990
year_of_death,1735,1861,1816,NaN,NaN
generation_name,Glorious,Compromise,Awakening,13th,Millennial
date_of_birth,"December 15, 1665","October 26, 1791",1710,12-Jul-61,5-Nov-90
place_of_birth,"Taunton, Bristol County, Massachusetts, United...","Berkshire, Berkshire, MA, United States","Berkley, Bristol County, Province of Massachus...","Flagstaff, AZ, United States","Beverly, MA, United States"
birth_longitude,41.9164,42.3931,41.826,35.1813,42.5437
birth_latitude,-71.1544,-73.5086,-71.0988,-111.643,-70.8628
date_of_death,"December 22, 1735 (70)","May 28, 1861 (69)",1816 (106),Not yet,Not yet
place_of_death,"Berkley, Bristol County, Province of Massachus...","Berkshire, Berkshire, MA, United States","Cooperstown, Otsego, N.Y.",Not anywhere,Not anywhere
death_longitude,41.826,42.3931,42.7002,NaN,NaN


In [20]:

generations_df.sample(5).T

generation_name,Glorious,13th,Parliamentary,Reformation,Silent
childhood_turning_entered,3,2,4,1,4
saeculum_name,New World,Millennial,Reformation,Reformation,Great Power
turnings_archetype,Hero,Nomad,Artist,Prophet,Artist
generations_archetype,Civic,Reactive,Adaptive,Idealist,Adaptive
birth_year_begin,1648,1961,1566,1483,1925
birth_year_end,1673,1981,1587,1511,1942
childhood_turning_description,"English Civil War, Republic, Restoration","Consciousness Revolution, Fourth Great Awakening",Armada Crisis,Tudor Renaissance,"Great Depression, World War II, Dust Bowl"
turning_year_begin,1649,1963,1569,1497,1929
turning_year_end,1675,1983,1594,1517,1946


In [21]:

turnings_df.sample(5).T

turning_name,Merrie England,The Transcendental Awakening,French & Indian Wars,The Era of Good Feelings,The Puritan Awakening
turning_number,1,2,3,1,2
turning_begin_year,1594,1822,1746,1794,1621
turning_end_year,1621,1844,1773,1822,1649
saeculum_name,New World,Civil War,Revolutionary,Civil War,New World
entering_elderhood,Reprisal,Republicans,Enlighteners,Liberty,Elizabethans
entering_midlife,Elizabethans,Compromisers,Awakeners,Republicans,Parliamentarians
entering_young_adulthood,Parliamentarians,Transcendentals,Liberty,Compromisers,Puritans
entering_childhood,Puritans,Gilded,Republicans,Transcendentals,Cavaliers
turning_notes,"Merrie England (First Turning, 1594–1621) was ...","The Transcendental Awakening (Second Turning, ...","French & Indian Wars (Third Turning, 1746–1773...","The Era of Good Feelings (First Turning, 1794–...","The Puritan Awakening (Second Turning, 1621–16..."


In [22]:

match_series = (generations_df['birth_year_begin'] >= min_year)
generations_df[match_series]['saeculum_name'].unique()

array(['Reformation', 'New World', 'Revolutionary', 'Civil War',
       'Great Power', 'Millennial'], dtype=object)

In [23]:

match_series = (turnings_df['turning_begin_year'] >= min_year)
turnings_df[match_series]['saeculum_name'].unique()

array(['Tudor', 'New World', 'Revolutionary', 'Civil War', 'Great Power',
       'Millennial'], dtype=object)

In [24]:

turnings_df['turning_end_year'].unique()

array([1459, 1487, 1517, 1542, 1569, 1594, 1621, 1649, 1675, 1704, 1727,
       1746, 1773, 1794, 1822, 1844, 1860, 1865, 1886, 1908, 1929, 1946,
       1964, 1984, 2008, 2029], dtype=int64)

In [25]:

columns_list = ['turning_begin_year', 'turning_end_year']
turnings_df[columns_list].apply(lambda row_series: int((row_series['turning_begin_year']+row_series['turning_end_year'])/2), axis=1)

turning_name
Retreat from France                    1447
The War of the Roses                   1473
The Tudor Renaissance                  1502
The Protestant Reformation             1529
Intolerance and Martyrdom              1555
The Armada Crisis                      1581
Merrie England                         1607
The Puritan Awakening                  1635
Reaction and Restoration               1662
The colonial Glorious Revolution       1689
The Augustan Age of Empire             1715
The Great Awakening                    1736
French & Indian Wars                   1759
American Revolution                    1783
The Era of Good Feelings               1808
The Transcendental Awakening           1833
The Mexican War & Sectionalism         1852
The Civil War                          1862
Reconstruction & Gilded Age            1875
The Third Great Awakening              1897
World War I & Prohibition              1918
The Great Depression & World War II    1937
The American High  

In [ ]:

logging.basicConfig(filename='label_arc.log', filemode='a', format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S', level=logging.DEBUG)
logging.info('Running Label Arc')

In [ ]:

logging.getLogger().setLevel(logging.CRITICAL)

In [28]:

show_stopped_babbitt_plot(cw_history_year_dict, cw_history_theta_dict, 2020)

In [ ]:

for stopped_year in range(min_year, 2031):
    show_stopped_babbitt_plot(cw_history_year_dict, cw_history_theta_dict, stopped_year)

In [ ]:

add_fashion_saeculum_image(1971)

In [ ]:

for stopped_year in range(min_year, 2031):
    add_fashion_saeculum_image(stopped_year)

In [ ]:

make_a_movie(2020, png_suffix='_saeculum_fashionable', movie_folder=saeculum_fashionable_movie_folder, movie_name='movie_to_2020.gif')

In [ ]:

make_a_movie(1989, png_suffix='_saeculum_fashionable', movie_folder=saeculum_fashionable_movie_folder, movie_name='stephen_movie.gif')

In [ ]:

min_decade = 10 * (min_year//10)
for i in range(min_decade, 2020, 10):
    url = 'https://en.wikipedia.org/wiki/{}s_in_fashion'.format(i)
    if exists(url):
        print('{} exists'.format(i))

In [ ]:

print(os.path.abspath(os.path.join(saeculum_movie_folder, 'plot_{}_saeculum.png'.format(2020))))

In [ ]:

# Pillow
print('{} -m pip install --upgrade Pillow'.format(sys.executable))
!{sys.executable} -m pip install --upgrade Pillow

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body;
var attribs = body.attributes;
var command = "notebook_path = " + "'"+attribs['data-notebook-path'].value+"'";
kernel.execute(command);

In [ ]:
# %load ../../load_magic/nbviewer.py

from IPython.display import HTML

notebook_viewer_url = 'https://nbviewer.jupyter.org/github/dbabbitt/notebooks/blob/master/'
notebook_viewer_url += '/'.join(notebook_path.split('/')[1:])
html_str = 'Click <a href="{}" target="_blank">here</a> to view notebook in nbviewer.'
HTML(html_str.format(notebook_viewer_url))